In [2]:
# !pip3 install --upgrade pandas
# !pip3 install --upgrade numpy
# !pip3 install --upgrade httplib2
# !pip3 install --upgrade instabot
# !pip3 install --upgrade google-api-python-client
# !pip3 install --upgrade vk-api

import json
from datetime import datetime, timedelta
import shutil
import pandas as pd
import numpy as np
import vk_api

from BT_library import GoogleSheets, DF_works, Vkontakte, Insta

from secrets import login, password, sobaka_spreadsheet_id, delta

In [2]:
range_name_vk_take_from_sheets = "Sheet1!A3:H"
range_to_put_vk_data = "Sheet1!A3:P"
range_to_put_LU = "Sheet1!E1:G1"

CREDSTOSERVICE = 'credentials.json'

sobakaru_vk_id = '-15911874'
active_scan = sobakaru_vk_id
count_of_executes = 1 # 12 is probably enough for 2 weeks of posting

In [6]:
vk = Vkontakte()
g = GoogleSheets() # r
d = DF_works()     # s

In [13]:
vk_session = vk.get_session(login, password)

wall, tools = vk.get_wall(vk_session, 
                          count_of_executes, active_scan)

parsed, postids = vk.parse_wall(vk, wall, delta)

morefeatures = vk.supply_post_with_more_data(vk_session, 
                                             postids, 
                                             active_scan)

service = g.auth(CREDSTOSERVICE)

posts_from_sheets = g.get_values(service, 
                                 range_name_vk_take_from_sheets, 
                                 sobaka_spreadsheet_id)

result = d.vk_compare(posts_from_sheets, parsed, morefeatures)

g.put_values(service, 
             range_to_put_vk_data, 
             sobaka_spreadsheet_id, 
             result)

g.put_last_updated(service, 
                   range_to_put_LU, 
                   sobaka_spreadsheet_id, 
                   delta)